In [4]:
import category_encoders
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [39]:
datos = pd.read_csv('pokemon-data.csv',delimiter =';')
y = datos.loc[:,'Tier'].copy()
X = datos.loc[:,datos.columns != 'Tier'].copy()
X

,Name,Types,Abilities,HP,Attack,Defense,Special Attack,Special Defense,Speed,Next Evolution(s),Moves
0,Abomasnow,"['Grass', 'Ice']","['Snow Warning', 'Soundproof']",90,92,75,92,85,60,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le..."
1,Abomasnow-Mega,"['Grass', 'Ice']",['Snow Warning'],90,132,105,132,105,30,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le..."
2,Abra,['Psychic'],"['Inner Focus', 'Magic Guard', 'Synchronize']",25,20,15,105,55,90,['Kadabra'],"['Teleport', 'Ally Switch', 'Barrier', 'Encore..."
3,Absol,['Dark'],"['Justified', 'Pressure', 'Super Luck']",65,130,60,75,60,75,[],"['Perish Song', 'Future Sight', 'Me First', 'R..."
4,Absol-Mega,['Dark'],['Magic Bounce'],65,150,60,115,60,115,[],"['Perish Song', 'Future Sight', 'Me First', 'R..."
...,...,...,...,...,...,...,...,...,...,...,...
913,Zubat,"['Flying', 'Poison']","['Infiltrator', 'Inner Focus']",40,45,35,30,40,55,['Golbat'],"['Absorb', 'Leech Life', 'Supersonic', 'Astoni..."
914,Zweilous,"['Dark', 'Dragon']",['Hustle'],72,85,70,65,70,58,['Hydreigon'],"['Double Hit', 'Dragon Rage', 'Focus Energy', ..."
915,Zygarde,"['Dragon', 'Ground']","['Aura Break', 'Power Construct']",108,100,121,81,95,95,"['Zygarde-10%', 'Zygarde-Complete']","['Glare', 'Bulldoze', 'Dragon Breath', 'Bite',..."
916,Zygarde-10%,"['Dragon', 'Ground']","['Aura Break', 'Power Construct']",54,100,71,61,85,115,[],"['Glare', 'Bulldoze', 'Dragon Breath', 'Bite',..."


In [40]:
class TierCathegorizer():
    def transform(self,y):
        y = pd.DataFrame(y)
        yaux = y.copy()
        tiers = np.where(np.logical_or(yaux.loc[:,'Tier'] == 'OU', yaux.loc[:,'Tier'] == 'Uber'),'bueno','malo')
        yaux.loc[:,'Tier'] = tiers
        return yaux

In [43]:
categorizador = TierCathegorizer()
y = categorizador.transform(y)
y

,Tier
0,malo
1,malo
2,malo
3,malo
4,malo
...,...
913,malo
914,malo
915,bueno
916,malo


In [44]:
# Creación de los conjuntos de test (10% de los ejemplos) y el resto de ejemplos
restoEjemplos_X, X_test, restoEjemplos_y, y_test =  train_test_split(X.loc[:,:],y.loc[:], test_size=0.10, random_state=123)

# Creación de los conjuntos de train (80% de los ejemplos) y de validación (resto de los ejemplos)
X_train, X_val, y_train, y_val =  train_test_split(restoEjemplos_X,restoEjemplos_y, train_size=0.80, test_size=0.20, random_state=123)
print(f"X_test: {X_test}, y_test: {y_test},X_train: {X_val}, y_train: {y_val}")

X_test:                 Name                 Types  \
349            Hoopa  ['Psychic', 'Ghost']   
653            Riolu          ['Fighting']   
7    Aegislash-Blade    ['Steel', 'Ghost']   
571          Pansear              ['Fire']   
171    Deoxys-Attack           ['Psychic']   
..               ...                   ...   
307  Gourgeist-Super    ['Ghost', 'Grass']   
498           Mewtwo           ['Psychic']   
50         Bastiodon     ['Rock', 'Steel']   
184            Ditto            ['Normal']   
814         Tornadus            ['Flying']   

                                     Abilities   HP  Attack  Defense  \
349                               ['Magician']   80     110       60   
653  ['Inner Focus', 'Prankster', 'Steadfast']   40      70       40   
7                            ['Stance Change']   60     150       50   
571                      ['Blaze', 'Gluttony']   50      53       48   
171                               ['Pressure']   50     180       20   
..     